First, import needed modules

In [2]:
import numpy as np
import pandas as pd
import os
from functools import reduce

Initalize variables

In [3]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import inkar dataset

In [6]:
inkar_raw = pd.read_csv(f'{path}/data/controls/inkar_2021.csv', encoding='UTF-8-SIG')

Rename columns

In [7]:
col_names = ['bereich', 'indikator', 'AVS', 'year', 'wert']
inkar_raw.columns = col_names
inkar_raw.columns

Index(['bereich', 'indikator', 'AVS', 'year', 'wert'], dtype='object')

Fix the AGS

In [8]:
inkar_raw['AVS'] = inkar_raw['AVS'].astype(str)
inkar_raw['AVS'] = np.where(inkar_raw['AVS'].str.len() < 8, '0' + inkar_raw['AVS'], inkar_raw['AVS'])

Convert Wert to numeric

In [9]:
inkar_raw['value'] = inkar_raw['wert'].str.replace(',','.').astype(float)

Filter df

In [10]:
vars_to_keep = ['Bevölkerung gesamt', 'Frauenanteil', 'Einwohnerdichte',  'Arbeitslose', 'Durchschnittsalter der Bevölkerung']
inkar_selected = inkar_raw[inkar_raw['indikator'].isin(vars_to_keep)]

Pivot df

In [11]:
inkar_pivot = pd.pivot(inkar_selected, index=['AVS', 'year'], columns='indikator', values='value')
inkar_pivot = inkar_pivot.reset_index()
inkar_pivot.shape

(106214, 7)

Rename columns

In [12]:
col_names = ['AVS', 'year', 'unemployed_inkar', 'pop_inkar', 'avg_age_inkar', 'pop_density_inkar', 'female_inkar']
inkar_pivot.columns = col_names

Change Verbandsschlüssel to AGS (2013)

In [13]:
avs_trans = pd.read_csv(f'{path}/data/avs_transition.csv', converters={'AGS': str, 'AVS': str})
inkar_controls = inkar_pivot.merge(avs_trans, on='AVS')
del inkar_controls['AVS']

Import Regionalstatistik datasets

In [73]:
# initialize dictionaries (we need to here, because the year does not uniquely identify the dfs)
income = {2007: 'Einkommen_2007.csv', 2010: 'Einkommen_2010.csv', 2013: 'Einkommen_2013.csv'}
# read in dfs
for year, file in income.items():
    income[year] = pd.read_csv(f'{path}/data/controls/{file}', encoding = 'ISO-8859-1', sep=';', converters={'AGS': str}, na_values=['-', 'x', '.'])
religion = pd.read_csv(f'{path}/data/controls/religion_2011.csv', encoding = 'ISO-8859-1', sep=';', converters={'AGS': str}, na_values=['-', 'x', '.'])
foreign = pd.read_csv(f'{path}/data/controls/nationalitaet_2011.csv', encoding = 'ISO-8859-1', sep=';', converters={'AGS': str}, na_values=['-', 'x', '.'])

Income: Fix and change AGS to 2013, rename columns and compute income per capita

In [54]:
for year, df in income.items():
    # fix AGS
    income[year]['AGS'] = income[year]['AGS'].astype(str)
    income[year]['AGS'] = np.where(income[year]['AGS'].str.len() < 6, income[year]['AGS'] + '0'*3, income[year]['AGS']) # add trailing 000 for kreisfreie Städte (AGS has 4-5 digits before, 7-8 after)
    income[year]['AGS'] = np.where(income[year]['AGS'].str.len() < 8, '0' + income[year]['AGS'], income[year]['AGS']) # add leading 0 for state ids < 10 (AGS has 7 digits)
    # read in AGS transition dataset
    ags_trans = pd.read_csv(f'{path}/data/ags_transition.csv', usecols=[f'AGS_{year}', 'AGS_2013'], converters={f'AGS_{year}': str, 'AGS_2013': str})
    # merge income[year] with ags trans
    income[year] = income[year].merge(ags_trans, left_on='AGS', right_on=f'AGS_{year}', how='left', indicator=True)
    print(income[year]._merge.value_counts())
    # delete old AGS and rename
    income[year]['AGS'] = np.where(income[year]['AGS_2013'].notna(), income[year]['AGS_2013'], income[year]['AGS'])
    income[year] = income[year].drop(['AGS_2013', f'AGS_{year}'], axis=1)
    # move AGS to front
    AGS = income[year].pop('AGS')
    income[year].insert(0, 'AGS', AGS)
    # rename columns
    col_names = ['AGS', 'year', 'GEN', 'taxable_persons', 'total_income', 'taxes', 'merge']
    income[year].columns = col_names
    # compute income per capita
    income[year]['income_pc'] = np.log(income[year]['total_income'] / income[year]['taxable_persons'] * 1000)
    # subset df
    income[year] = income[year][['AGS', 'year', 'income_pc']]

both          14846
left_only      2121
right_only        0
Name: _merge, dtype: int64
both          16464
left_only      2412
right_only        0
Name: _merge, dtype: int64
both          16465
left_only      2691
right_only        0
Name: _merge, dtype: int64


Concatenate income dfs

In [55]:
income = pd.concat(income.values())
income.head()

,AGS,year,income_pc
0,00000,2007,10.362778
1,01000,2007,10.362917
2,01001000,2007,10.219578
3,01002000,2007,10.198012
4,01003000,2007,10.231703


Pivot religion df

In [74]:
religion = pd.pivot_table(religion, index=['AGS'], columns=['Religion'], values='Bevölkerung', aggfunc=np.sum, fill_value=0)
religion.rename(columns = {'Römisch-katholische Kirche': 'Catholic'}, inplace = True)
religion = religion.reset_index()
religion = religion[['AGS', 'Catholic']]
religion.head()

Religion,AGS,Catholic
0,1,167565
1,10,619696
2,1001,5220
3,1002,17368
4,1003,17793


Since religion and foreign data is only available in 2011

In [75]:
df_year = {2008: '', 2009: '', 2010: '', 2012:'', 2013:'', 2014:''}
for year in df_year.keys(): 
    df_year[year] = religion
    df_year[year]['year'] = year
    #print(df_year[year]['year'].value_counts())
religion = pd.concat(df_year.values())


In [71]:
dfs = [religion, foreign]
for df in dfs:
    df_year = {2008: '', 2009: '', 2010: '', 2012:'', 2013:'', 2014:''}
    for year in df_year.keys(): 
        df_year[year] = df
        df_year[year]['year'] = year
        print(df_year[year].columns)
    df = pd.concat(df_year.values())
religion, foreign = dfs

Index(['AGS', 'Catholic', 'year'], dtype='object', name='Religion')
Index(['AGS', 'Catholic', 'year'], dtype='object', name='Religion')
Index(['AGS', 'Catholic', 'year'], dtype='object', name='Religion')
Index(['AGS', 'Catholic', 'year'], dtype='object', name='Religion')
Index(['AGS', 'Catholic', 'year'], dtype='object', name='Religion')
Index(['AGS', 'Catholic', 'year'], dtype='object', name='Religion')
Index(['AGS', 'Bevölkerung', 'weiblich', 'Ausländer/-innen', 'year'], dtype='object')
Index(['AGS', 'Bevölkerung', 'weiblich', 'Ausländer/-innen', 'year'], dtype='object')
Index(['AGS', 'Bevölkerung', 'weiblich', 'Ausländer/-innen', 'year'], dtype='object')
Index(['AGS', 'Bevölkerung', 'weiblich', 'Ausländer/-innen', 'year'], dtype='object')
Index(['AGS', 'Bevölkerung', 'weiblich', 'Ausländer/-innen', 'year'], dtype='object')
Index(['AGS', 'Bevölkerung', 'weiblich', 'Ausländer/-innen', 'year'], dtype='object')


In [56]:
foreign.columns

Index(['AGS', 'Bevölkerung', 'weiblich', 'Ausländer/-innen'], dtype='object')

Concatenate datasets

In [161]:

rs_controls = rs_controls.sort_values(['AGS', 'year'])
rs_controls.head()

,AGS,Arbeitslose,year,Einkünfte je Steuerpflichtigen in Tsd.,Bevölkerung,weiblich,Ausländer/-innen,Catholic,avg. age
1,01001000,4948.0,2008,NaN,NaN,NaN,NaN,NaN,NaN
1,01001000,NaN,2010,27,NaN,NaN,NaN,NaN,NaN
1,01001000,NaN,2011,NaN,82258.0,41724.0,5129.0,NaN,NaN
51,01001000,NaN,2011,NaN,NaN,NaN,NaN,5220.0,NaN
1,01001000,5127.0,2012,NaN,NaN,NaN,NaN,NaN,NaN


Rename columns

In [182]:
col_names = ['AGS', 'unemployed_rs', 'year', 'income_rs', 'pop_rs', 'female_rs', 'foreign_rs', 'catholic_rs', 'avg_age_rs']
rs_controls.columns = col_names

Load BBSR datasets

In [109]:
files = ['ref-gemeinden-1990-2000.xlsx', 'ref-gemeinden-2000-2010.xlsx', 'ref-gemeinden-2010-2020.xlsx']
bbsr_dfs = {}
for year in range(1990, 2000):
   bbsr_dfs[year] = pd.read_excel(f'{path}/data/controls/{files[0]}', sheet_name=f'{year}-{year+1}', header=1, decimal = ',')
for year in range(2000, 2010):
   bbsr_dfs[year] = pd.read_excel(f'{path}/data/controls/{files[1]}', sheet_name=f'{year}-{year+1}', header=1, decimal = ',')
for year in range(2010, 2020):
    if year < 2016:
       bbsr_dfs[year] = pd.read_excel(f'{path}/data/controls/{files[2]}', sheet_name=f'{year}-{year+1}', header=1, decimal = ',')
    else: # header changed
       bbsr_dfs[year] = pd.read_excel(f'{path}/data/controls/{files[2]}', sheet_name=f'{year}-{year+1}', header=0, decimal = ',')

Rename and filter columns, Fix the AGS, insert year column

In [111]:
filtered_bbsr = {}
for year, df in bbsr_dfs.items():
    # for 1990-1996 there  is only data on the area and population
    if year <= 1996:
        col_names = ['AGS', 'Name', 'flächenprop.', 'bevölkerungsprop.', 'area', 'pop_bbsr', 'Kennziffer.1', 'Name.1']
        bbsr_dfs[year].columns = col_names
        filtered_bbsr[year] = bbsr_dfs[year][[f'AGS', 'area', 'pop_bbsr']]
    # for 1996-2020 there is also data on the number of employed
    elif year <= 2020:
        col_names = ['AGS', 'Name', 'flächenprop.', 'bevölkerungsprop.', 'beschäftigtenprop.', 'area', 'pop_bbsr', 'employed_bbsr', 'Kennziffer.1', 'Name.1']
        bbsr_dfs[year].columns = col_names
        filtered_bbsr[year] = bbsr_dfs[year][[f'AGS', 'area', 'pop_bbsr', 'employed_bbsr']]
    # insert year
    filtered_bbsr[year]['year'] = year
    # fix AGS
    filtered_bbsr[year]['AGS'] = filtered_bbsr[year]['AGS'].astype(str)
    filtered_bbsr[year]['AGS'] = np.where(filtered_bbsr[year]['AGS'].str.len() < 8, '0' + filtered_bbsr[year]['AGS'], filtered_bbsr[year]['AGS'])

/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_46242/4007496148.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_bbsr[year]['year'] = year
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_46242/4007496148.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_bbsr[year]['AGS'] = filtered_bbsr[year]['AGS'].astype(str)
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_46242/4007496148.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

Convert AGS to 2013

In [112]:
for year, df in filtered_bbsr.items():
    # skip 2013
    if year == 2013:
        continue
    # read in AGS transition dataset
    ags_trans = pd.read_csv(f'{path}/data/ags_transition.csv', usecols=[f'AGS_{year}', 'AGS_2013'], converters={f'AGS_{year}': str, 'AGS_2013': str})
    # merge filtered_bbsr[year] with ags trans
    filtered_bbsr[year] = filtered_bbsr[year].merge(ags_trans, left_on='AGS', right_on=f'AGS_{year}', how='left')
    # delete old AGS and rename
    filtered_bbsr[year] = filtered_bbsr[year].drop(['AGS', f'AGS_{year}'], axis=1)
    filtered_bbsr[year].rename(columns = {'AGS_2013':'AGS'}, inplace = True)
    # move AGS to front
    AGS = filtered_bbsr[year].pop('AGS')
    filtered_bbsr[year].insert(1, 'AGS', AGS)

Concatenate all BBSR dfs

In [113]:
bbsr_controls = pd.concat(filtered_bbsr.values())

Concatenate INKAR, RS and BBSR dfs

In [183]:
all_controls = pd.concat([inkar_controls, rs_controls, bbsr_controls])
#all_controls = all_controls.set_index(['AGS', 'year'])
all_controls.shape

(966798, 17)

In [184]:
grouped_controls = all_controls.groupby(['AGS', 'year'], as_index=False).first()
grouped_controls.shape

(337050, 17)

Combine variables

In [194]:
controls = grouped_controls.copy()
# Population: Inkar has exact number from 95-17, BBSR has in thousands -94, in hundreds from 18 (despite header saying it is in thousands)
controls['pop'] = controls['pop_inkar']
controls['pop'] = np.where(controls['year'] < 1995, controls['pop_bbsr'] * 1000, controls['pop'])
controls['pop'] = np.where(controls['year'] > 2017, controls['pop_bbsr'] * 100, controls['pop'])
# pop density can now be calculated since we have area and population for all years
controls['pop_density'] = controls['pop'] / controls['area']
# share of unemployed
controls['unemployed'] = controls['unemployed_inkar'] / controls['pop'] * 100
# share of foreigners
controls['foreign'] = controls['foreign_rs'] / controls['pop'] * 100
# catholics
controls['catholic'] = controls['catholic_rs'] / controls['pop'] * 100
# avg_age: inkar has more data
controls['avg_age'] = controls['avg_age_inkar']
# same for share of females
controls['female'] = controls['female_inkar']
# take the log of income * 10000
controls['income'] = controls['income_rs'].str.replace(',', '.').astype(float)*1000
controls['income'] = np.log(controls['income'])

In [196]:
controls = controls[['AGS', 'year', 'pop_density', 'unemployed', 'foreign', 'catholic', 'avg_age', 'female', 'income']]
controls.head()

,AGS,year,pop_density,unemployed,foreign,catholic,avg_age,female,income
0,01001000,1990,1536.550745,NaN,NaN,NaN,NaN,NaN,NaN
1,01001000,1991,1547.196593,NaN,NaN,NaN,NaN,NaN,NaN
2,01001000,1992,1550.850461,NaN,NaN,NaN,NaN,NaN,NaN
3,01001000,1993,1559.000709,NaN,NaN,NaN,NaN,NaN,NaN
4,01001000,1994,1558.114812,NaN,NaN,NaN,NaN,NaN,NaN


Export dataset

In [197]:
controls = controls.set_index(['AGS', 'year'])
controls.to_csv(f'{path}/data/controls.csv', encoding = 'utf-8-sig')

In [5]:
controls = pd.read_csv(f'{path}/data/controls.csv', encoding = 'utf-8-sig')